In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import os
import cv2
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset

In [4]:


class KeypointDataset(Dataset):
    def __init__(self, img_dir, data_file):
        self.img_dir = img_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]
        img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(img)
        keypoints = np.array(items["kps"]).flatten()
        keypoints = keypoints.astype(np.float32)
        keypoints[::2] *= 224.0 / w # x
        keypoints[1::2] *= 224.0 / h # y

        return img, keypoints


In [ ]:
train_dataset = KeypointDataset("data/images", "data/data_train.json")
val_dataset = KeypointDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2) # 14 keypoints, 2 coordinates each

model = model.to(device)

# Training

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
epochs = 50
for epoch in range (epochs):
    for i, (images, keypoints) in enumerate(train_loader):
        images = images.to(device)
        keypoints = keypoints.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, keypoints)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, step {i}, loss {loss.item()}")

In [ ]:
torch.save(model.state_dict(), "keypoints_model.pth")